# This notebook is a practice for DMP #

Step 1: check out what dialogs are available from the dialogs package.

In [10]:
import sipparty.sip.dialogs  
sipparty.sip.dialogs.AllDialogsTypes

[sipparty.sip.dialogs.call.SimpleCall]

A `Dialog` instance is our interface to another person ("User Agent" in SIP terms) talking in SIP. Using instances of `Dialog` we can invite other people to talk to us, and end calls when we're done.

> Note: `sipparty` can use instances of `Dialog` for any SIP session, including registrations, which are covered later.

Step 2: Create two `SimpleCall` dialogs, one for each side of the call. Try converting one of the dialogs to a string using `print(d1)` or `str(d1)`. You'll see that it is described by a Finite State Machine, which is simply a way of tracking the state (such as "not yet in a call", "starting a call" etc.) when we give it certain inputs (such as "start a call to person Y", or "hang up the call").

In [11]:
from sipparty.sip.dialogs import SimpleCall
d1 = SimpleCall()
d2 = SimpleCall()
print(d1)

'SimpleCall' 'SimpleCall3':
  'TerminatingDialog':
    'receiveResponse2' -> 'Terminated'

  'Initial':
    'initiate' -> 'InitiatingDialog'
    'receiveRequestINVITE' -> 'InDialog'

  'InitiatingDialog':
    'receiveResponse2' -> 'InDialog'
    'receiveResponse4' -> 'Error'
    'receiveResponse18' -> 'InitiatingDialog'

  'InDialog':
    'terminate' -> 'TerminatingDialog'
    'receiveResponse2' -> 'InDialog'
    'receiveRequestBYE' -> 'Terminated'

Current state: 'Initial'


Step 3: See what inputs we can give to the dialog.

In [12]:
d1.Inputs

{'initiate',
 'receiveRequestBYE',
 'receiveRequestINVITE',
 'receiveResponse18',
 'receiveResponse2',
 'receiveResponse4',
 'terminate'}

Try to initiate a dialog.

In [13]:
d1.hit('initiate')

ERROR:sipparty.fsm.fsm:Hit exception processing FSM action <function weak_action_sendRequestINVITE at 0x108ef1b18>.
Traceback (most recent call last):
  File "sipparty/fsm/fsm.py", line 625, in _fsm_hit
    action(*args, **kwargs)
  File "sipparty/fsm/fsm.py", line 470, in weak_action
    srv = func(*args, **kwargs)
  File "sipparty/util.py", line 697, in weak_method
    return getattr(sr, method)(*pass_args, **pass_kwargs)
  File "sipparty/sip/dialog.py", line 222, in sendRequest
    reqdAttr, self.__class__.__name__))
ValueError: Attribute 'fromURI' of 'SimpleCall' instance required to send a request is None.


Ah, looks like we need to give the dialog a bit more information before we can start a call. We need to set the `fromURI` property. A SIP URI identifies a User Agent's SIP identity. In its simplest form a SIP URI looks like `sip:username@host`. Try setting that as the from URI.  

In [5]:
d1.fromURI = 'sip:username@host'
print(d1.fromURI)
print(repr(d1.fromURI))

sip:username@host
URI(headers='', absoluteURIPart=None, scheme='sip', aor=AOR(username='username', host=Host(port=None, address='host')), parameters='')


We've printed the `str()` conversion of the URI, which looked as we expected. But look what happened when we printed the `repr()` conversion. The URI is broken out into its constituent parts.

> NB: for the interested the complete form of the URI is defined in section @@@ of [RFC3261](https://tools.ietf.org/html/rfc3261).

Let's try that call again.

In [6]:
d1.hit('initiate')

ERROR:sipparty.fsm.fsm:Hit exception processing FSM action <function weak_action_sendRequestINVITE at 0x108e4ba28>.
Traceback (most recent call last):
  File "sipparty/fsm/fsm.py", line 625, in _fsm_hit
    action(*args, **kwargs)
  File "sipparty/fsm/fsm.py", line 470, in weak_action
    srv = func(*args, **kwargs)
  File "sipparty/util.py", line 697, in weak_method
    return getattr(sr, method)(*pass_args, **pass_kwargs)
  File "sipparty/sip/dialog.py", line 222, in sendRequest
    reqdAttr, self.__class__.__name__))
ValueError: Attribute 'toURI' of 'SimpleCall' instance required to send a request is None.


Looks like we need a toURI as well. But since we're going to be calling `d2`, we can set all the remaining parameters up at the same time.

In [9]:
d1.toURI = d2.fromURI = 'sip:username2@host'
d1.initiate()

ERROR:sipparty.fsm.fsm:Bad input 'initiate' to 'SimpleCall' instance 'SimpleCall1' (current state 'InitiatingDialog').


UnexpectedInput: Bad input 'initiate' to 'SimpleCall' instance 'SimpleCall1' (current state 'InitiatingDialog').